ą# Prezentacja o bazie NoSQL Qdrant

---

## Autorzy

- Mateusz Malich, Aleksander Batko

---

## Bibliografia

- Oficjalna strona Qdrant: [https://qdrant.tech](https://qdrant.tech)
- Repozytorium GitHub Qdrant: [https://github.com/qdrant/qdrant](https://github.com/qdrant/qdrant)
- Artykuły i blogi techniczne o Qdrant (np. Medium, Towards Data Science)
- Dokumentacja Qdrant: [https://qdrant.tech/documentation/](https://qdrant.tech/documentation/)
- Baza wektorowa: [https://learn.microsoft.com/pl-pl/fabric/real-time-intelligence/vector-database](https://learn.microsoft.com/pl-pl/fabric/real-time-intelligence/vector-database)

---

## Podstawowe informacje o bazie i jej historii

**Qdrant** to nowoczesna baza danych NoSQL typu **vector database**, zaprojektowana do przechowywania i szybkiego wyszukiwania wektorów osadzania (embeddings) generowanych przez modele AI.

### Cechy Qdrant:
- Wysoka wydajność wyszukiwania milionów wektorów przy niskim opóźnieniu
- Obsługa metadanych do zaawansowanego filtrowania wyników
- Prosty REST API i gRPC do integracji
- Możliwość tworzenia i usuwania kolekcji w czasie działania
- Open Source na licencji Apache 2.0
- Dostępna lokalnie (on-premises) oraz jako chmurowa usługa Qdrant Cloud

### Historia:
- Projekt rozpoczęty w 2021 roku przez Qdrant GmbH z Berlina
- Publiczne wydanie open source w 2022
- Dynamiczny rozwój w latach 2023-2024 z naciskiem na wydajność i łatwość użycia

---

### Czym jest baza wektorowa?
Baza wektorowa to specjalny rodzaj bazy danych zaprojektowany do przechowywania i szybkiego wyszukiwania danych w postaci wektorów liczbowych (np. list liczb). Wektory te często powstają jako reprezentacje (embeddingi) obiektów takich jak teksty, obrazy, dźwięki czy inne dane, które dzięki temu można porównywać pod kątem ich znaczenia lub podobieństwa.

## Dlaczego to ważne?
Tradycyjne bazy danych świetnie radzą sobie z danymi strukturalnymi i wyszukiwaniem po kluczach lub pełnotekstowym. Jednak gdy chcemy wyszukiwać np. „podobne obrazy” lub „teksty o podobnym sensie”, potrzebujemy porównywać dane w przestrzeni wielowymiarowej. Do tego służą bazy wektorowe.

## Praktyczny przykład
- Obiekt (np. zdanie) jest zamieniany na wektor liczb (embedding) za pomocą modelu AI
- Wektory umieszczane są w bazie
- Podczas wyszukiwania podajesz zapytanie, które również jest przekształcane na wektor
- Baza szybko znajduje wektory najbardziej „podobne” do zapytania, np. na podstawie odległości kosinusowej lub euklidesowej
---

### Firmy korzystające z Qdrant:

- Microsoft
- Disney dla Disney+
- Kaufland
- X (Twitter)
- Discord

## Instalacja i dostępność

### Instalacja lokalna

Na początku trzeba pobrać najnowszy obraz Qdrant z Dockerhuba:

```bash
docker pull qdrant/qdrant
```bash
Potem uruchom jako usługe:

```bash
docker run -p 6333:6333 -p 6334:6334 \
    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
    qdrant/qdrant
```bash

# Potrzebne importy oraz nasze skrypty

In [34]:
import os
from dotenv import load_dotenv
import glob
load_dotenv()
from qdrant import QdrantDB
from vector import get_openai_embedding
import uuid


# Klasa służąca do komunikacji z bazą danych


```python

class QdrantDB:
    def __init__(self, url: str = "localhost", port: int = 6333, api_key: Optional[str] = None):
        """
        Initialize the Qdrant client.

        Args:
            url: URL of the Qdrant server
            port: Port of the Qdrant server
            api_key: API key for authentication (if needed)
        """
        if api_key:
            self.client = QdrantClient(url=url, port=port, api_key=api_key, check_compatibility=False)
        else:
            self.client = QdrantClient(url=url, port=port, check_compatibility=False)
```






In [35]:
qdrant_client = QdrantDB(url="localhost", port=6333)

# Tworzenie kolekcji

In [36]:
collection_name = "notatki"
vector_size = 3072  # Rozmiar modelu
success = qdrant_client.create_collection(
    collection_name=collection_name,
    vector_size=vector_size,
    distance="Cosine"
)
print(f"Collection created: {success}")

Error creating collection: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `notatki` already exists!"},"time":0.00063775}'
Collection created: False


# Lista dostępnych kolekcji

In [37]:
collections = qdrant_client.list_collections()
print("Available collections:")
for collection in collections:
    print(f"- {collection.name}")

Available collections:
- reports
- aidevs3
- notes
- notatki
- my_collection


# Dodawanie dokumentów do kolekcji

```python
    def create_collection(self, collection_name: str, vector_size: int, distance: str = "Cosine"):
        """
        Create a new collection in Qdrant.

        Args:
            collection_name: Name of the collection
            vector_size: Dimensionality of vectors
            distance: Distance metric ("Cosine", "Euclid", or "Dot")

        Returns:
            True if collection was created successfully
        """
        try:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=models.VectorParams(
                    size=vector_size,
                    distance=distance
                )
            )
            return True
        except Exception as e:
            print(f"Error creating collection: {e}")
            return False
```

In [38]:
notes_directory = "Documents/pages"

markdown_files = glob.glob(os.path.join(notes_directory, "*.md"))

print(f"Found {len(markdown_files)} markdown files")

for file_path in markdown_files:
    try:
        filename = os.path.basename(file_path)

        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        vector = get_openai_embedding(content)

        payload = {
            "filename": filename,
            "source_path": file_path
        }

        file_id = str(uuid.uuid4())

        success = qdrant_client.add_point(
            collection_name=collection_name,
            id=file_id,
            vector=vector,
            payload=payload
        )

        if success:
            print(f"Added file '{filename}' to collection with ID '{file_id}'")
        else:
            print(f"Failed to add file '{filename}'")

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

Found 67 markdown files
Added file 'Concept drift.md' to collection with ID '3cab82a1-ac5b-4f30-827b-6f84147a0031'
Added file 'Swift.md' to collection with ID 'cc4e282b-06f6-463f-ae19-3c13b215c37c'
Added file 'Generative AI as Code Generators.md' to collection with ID '47b927b4-f20c-4f53-8885-e73eb1efb0b6'
Added file 'Prompt Engineering Guide.md' to collection with ID '600b100c-9e53-4495-ab8d-03c12a6ea792'
Added file 'LLM's and GAI tools.md' to collection with ID 'bd3c9717-55a5-402c-b271-2458312b1663'
Added file 'Kotlin.md' to collection with ID 'b95c1d65-f8d3-4dc7-bc71-6196015930bb'
Added file 'Confusion matrix.md' to collection with ID '1d067386-5439-4726-926d-f4be9e941549'
Added file 'Application Programming Interface.md' to collection with ID '68ec5788-f2d2-447e-8400-a325b9e984bd'
Added file 'Prompt Evaluation.md' to collection with ID 'f38a48d5-078a-4277-b50e-35333a8629ee'
Added file 'Sbody.md' to collection with ID '39b679a5-90c7-403e-93d9-a75994a20a5c'
Added file 'Regularized re

# Wyszukiwanie dokumentów w kolekcji

```python

vector = get_openai_embedding(args.text) # wazane bo zapytanie tez musi być w formie wektora
    def search(self, collection_name: str, query_vector: List[float], limit: int = 10,
               filter: Optional[models.Filter] = None):
        """
        Search for similar vectors in the collection.

        Args:
            collection_name: Name of the collection
            query_vector: Vector to search for
            limit: Maximum number of results
            filter: Optional filter conditions

        Returns:
            List of search results with points
        """
        try:
            return self.client.search(
                collection_name=collection_name,
                query_vector=query_vector,
                limit=limit,
                query_filter=filter
            )
        except Exception as e:
            print(f"Error searching: {e}")
            return []
```

In [39]:
query_text = "AI and machine learning technologies"
query_vector = get_openai_embedding(query_text)

search_results = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3
)
print(f"\nSearch results for: '{query_text}'")
for result in search_results:
    print(f"ID: {result.id}, Score: {result.score}")
    if hasattr(result, 'payload') and result.payload:
        print(f"Text: {result.payload.get('filename', 'N/A')}")
    print("---")


Search results for: 'AI and machine learning technologies'
ID: daac0c06-c49c-4965-a56f-ed56523b3929, Score: 0.6133485
Text: Artificial Intelligence.md
---
ID: f719f9ab-d51e-4598-a9db-dd7f06653d95, Score: 0.6133485
Text: Artificial Intelligence.md
---
ID: 99275c04-dc23-43ae-a497-153f7872a781, Score: 0.56131697
Text: Machine Learning.md
---


# Informacje o konkretnym punkcie

In [40]:
point = qdrant_client.get_point(collection_name=collection_name, id="aa4edb9c-4359-431a-9169-c9be003d61f3")
print(f"\nRetrieved point by ID:")
if point and hasattr(point, 'payload'):
    print(f"Text: {point.payload.get('filename', 'N/A')}")



Retrieved point by ID:
Text: Generative Artificial Intelligence.md


# Aktualizacja punktu

Aktualizacja punktu również wymaga zmiany wektora [Qdrant API - Upsert Points](https://api.qdrant.tech/api-reference/points/upsert-points)


In [41]:
id = "aa4edb9c-4359-431a-9169-c9be003d61f3"
updated_payload = {
    "category": "artificial intelligence",
    "importance": "high"
}
success = qdrant_client.update_payload(
    collection_name=collection_name,
    id=id,
    payload=updated_payload
)

success = qdrant_client.add_point(
    collection_name=collection_name,
    id=id,
    vector=[0.1, 0.2, 0.3],  # Example vector, should be the same size as the original
    payload=updated_payload
)

print(f"Payload updated: {success}")

Error adding points: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Wrong input: Vector dimension error: expected dim: 3072, got 3"},"time":0.000614042}'
Payload updated: False


# Liczebnośc kolekcji

In [42]:
count = qdrant_client.count_points(collection_name=collection_name)
print(f"\nTotal points in collection: {count}")


Total points in collection: 134
